***GENERATED CODE FOR AcquisitionsClassifier PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_transform"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    indexed = indexed.drop(feature).withColumnRenamed(outcol, feature)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        feature).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            feature, dfReturn[feature].cast(IntegerType()))
        return changed_type_df
    # changed_type_df.show(3)
    return dfReturn


Feature_Transformations_Methods = {
    "String Indexer": StringIndexerTransform,

}


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, spark, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        for transformation in featureData:
            feature = transformation["feature"]
            if transformation["transformation"] != '' and transformation["selected"].lower() == "true" and not (feature.__contains__("_transform")):
                transformationDF = Feature_Transformations_Methods["%s" % transformation["transformation"]](
                    transformationDF, transformation)

        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    df.columns.intersection(listOfFeatures)
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=15,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	AcquisitionsClassifier_DBFS = DBFSConnector.fetch([], {}, "5eac6afd86c504fa4f862f5a", spark, "{'url': '/Demo/CustomerAcquisitionTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	AcquisitionsClassifier_AutoFE = TransformationMain.run(AcquisitionsClassifier_DBFS, spark,json.dumps( {"FE": [{"transformationsData": {"feature_label": "City"}, "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "ARVADA", "max": "WHEAT RIDGE", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Product_Category"}, "feature": "Product_Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "Furniture", "max": "Technology", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Product_Sub-Category"}, "feature": "Product_Sub-Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "Appliances", "max": "Telephones and Communication", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Count", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2416", "mean": "1.08", "stddev": "0.3", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": {}, "feature": "Customer ID", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2416", "mean": "1753.54", "stddev": "945.6", "min": "2", "max": "3402", "missing": "0"}}, {"transformationsData": {}, "feature": "Store Number", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2416", "mean": "102.38", "stddev": "1.81", "min": "100", "max": "105", "missing": "0"}}, {"transformationsData": {"feature_label": "Customer Segment"}, "feature": "Customer Segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "Consumer", "max": "Small Business", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "First Name"}, "feature": "First Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "AAXXX", "max": "ZEXXX", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Last Name"}, "feature": "Last Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "ABXXXX", "max": "ZIXXXXXXX", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Address"}, "feature": "Address", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "10 MEADOW ROSE LN", "max": "9924 W ARLINGTON AVE", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "State"}, "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "CO", "max": "CO", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Zip", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2416", "mean": "80112.23", "stddev": "102.27", "min": "80002", "max": "80602", "missing": "0"}}, {"transformationsData": {}, "feature": "DriveTime", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "2416", "mean": "13.07", "stddev": "7.29", "min": "0.65", "max": "30.0", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "Length of Residense", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2416", "mean": "11.04", "stddev": "10.82", "min": "0", "max": "55", "missing": "0"}}, {"transformationsData": {}, "feature": "MOR BANK: UPSCALE MERCHANDISE BUYER", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2416", "mean": "0.01", "stddev": "0.15", "min": "0", "max": "4", "missing": "0"}}, {"transformationsData": {"feature_label": "MOSAIC HOUSEHOLD"}, "feature": "MOSAIC HOUSEHOLD", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "A01", "max": "U00", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "MOSAIC DESCRIPTION"}, "feature": "MOSAIC DESCRIPTION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "Aspirational Fusion", "max": "Young City Solos", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Customer_Lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "2416", "mean": "-104.92", "stddev": "0.12", "min": "-105.226518", "max": "-104.65312", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "Customer_Lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "2416", "mean": "39.69", "stddev": "0.09", "min": "39.433416", "max": "39.998058", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "Store ID", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2416", "mean": "102.38", "stddev": "1.81", "min": "100", "max": "105", "missing": "0"}}, {"transformationsData": {}, "feature": "Store_Lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "2416", "mean": "-104.92", "stddev": "0.12", "min": "-105.077754", "max": "-104.717928", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "Store_Lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "2416", "mean": "39.7", "stddev": "0.08", "min": "39.565799", "max": "39.856274", "missing": "0"}, "transformation": ""}, {"transformationsData": {"feature_label": "Channel"}, "feature": "Channel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2416", "mean": "", "stddev": "", "min": "Catalog", "max": "eCommerce", "missing": "0"}, "transformation": "String Indexer"}, {"feature": "City_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "2416", "mean": "1.58", "stddev": "2.26", "min": "0.0", "max": "17.0", "missing": "0"}}, {"feature": "Product_Category_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "2416", "mean": "0.63", "stddev": "0.79", "min": "0", "max": "2", "missing": "0"}}, {"feature": "Product_Sub-Category_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "2416", "mean": "5.08", "stddev": "4.27", "min": "0.0", "max": "16.0", "missing": "0"}}, {"feature": "Customer Segment_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "2416", "mean": "1.24", "stddev": "1.11", "min": "0", "max": "3", "missing": "0"}}, {"feature": "First Name_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "2416", "mean": "78.28", "stddev": "78.08", "min": "0.0", "max": "343.0", "missing": "0"}}, {"feature": "Last Name_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "2416", "mean": "153.27", "stddev": "141.4", "min": "0.0", "max": "556.0", "missing": "0"}}, {"feature": "Address_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "2416", "mean": "383.53", "stddev": "308.95", "min": "0.0", "max": "1124.0", "missing": "0"}}, {"feature": "State_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "2416", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}}, {"feature": "MOSAIC HOUSEHOLD_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "2416", "mean": "15.12", "stddev": "13.37", "min": "0.0", "max": "59.0", "missing": "0"}}, {"feature": "MOSAIC DESCRIPTION_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "2416", "mean": "5.85", "stddev": "4.63", "min": "0.0", "max": "19.0", "missing": "0"}}, {"feature": "Channel_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "2416", "mean": "0.49", "stddev": "0.68", "min": "0", "max": "2", "missing": "0"}}]}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	functionClassification(AcquisitionsClassifier_AutoFE, ["City", "Product_Category", "Product_Sub-Category", "Count", "Customer ID", "Store Number", "Customer Segment", "First Name", "Last Name", "Address", "State", "Zip", "DriveTime", "Length of Residense", "MOR BANK: UPSCALE MERCHANDISE BUYER", "MOSAIC HOUSEHOLD", "MOSAIC DESCRIPTION", "Customer_Lon", "Customer_Lat", "Store ID", "Store_Lon", "Store_Lat"], "Channel")

except Exception as ex: 
	logging.error(ex)
